First import the things that we need from the pipeline code

In [1]:
import numpy as np
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Import my tools code from https://github.com/drlaw1558/jwst/

In [2]:
import mirim_tools as mirim_tools

Make a new CDP-7b reference file with all relevant pixel origin transforms. Read it in and use to do a test transform.

In [3]:
import mirim_createwcsfiles_cdp7b as mirim_createwcsfiles_cdp7b

Point to the IDT-delivered CDP file that we're using

In [4]:
cdpfile="/Users/dlaw/jwst-drl/trunk/cdp/cdp7b/MIRI_FM_MIRIMAGE_DISTORTION_7B.03.00.fits"

Specify our output filenames for the two asdf files

In [5]:
distfile="out/jwst_miri_imager_distortion_cdp7b.asdf"

In [6]:
offfile="out/jwst_miri_filteroffset_cdp7b.asdf"

Call the code to actually make the reference files

In [7]:
mirim_createwcsfiles_cdp7b.make_references(cdpfile,{'DISTORTION': distfile, 'FILTEROFFSET':offfile})

Now point to these reference files and load the transform into the function 'xform'

In [8]:
refs = {"distortion": distfile, "filteroffset": offfile}

In [9]:
xform=mirim_tools.imager_detector_to_v2v3_cdp7b('F770W',refs)

The pipeline uses 0-indexed detector pixels; as of CDP7b the MIRI ASDF object assumes this form of input and converts as necessary to the reference frames needed by various IDT transforms.  The reference point of the imager is 692.5, 511.5 in this 0-indexed convention; test that it gives the correct v2,v3

In [15]:
x,y=692.5,511.5

In [16]:
v2,v3=xform(x,y)

In [17]:
print(v2,v3)

(-453.3633633490669, -374.0686305853809)


And the inverse transform

In [18]:
xform.inverse(-453.3633633490669, -374.0686305853809)

(692.496064329321, 511.4999501761038)

This location (-453.363, -374.0686) is the agreed-upon reference point. 
Now run a full test of the transform against a table of known values using a dedicated testing script.  If no output, it worked.

In [19]:
mirim_createwcsfiles_cdp7b.test_transform(refs)

As of Apr 12 2017 we can't test the bounding box implementation directly because it is build into the pipeline, not the model class.  Therefore we must test this by actually running the pipeline on MIRI Imager test and reading the transform in from there.  This is done with the mirim_imagertest.ipynb notebook